In [12]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
import matplotlib.pyplot as plt
import math

In [13]:
ds = pd.read_csv('Admission_Predict.csv') 

In [14]:
ds.tail(10)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
390,391,314,102,2,2.0,2.5,8.24,0,0.64
391,392,318,106,3,2.0,3.0,8.65,0,0.71
392,393,326,112,4,4.0,3.5,9.12,1,0.84
393,394,317,104,2,3.0,3.0,8.76,0,0.77
394,395,329,111,4,4.5,4.0,9.23,1,0.89
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67
399,400,333,117,4,5.0,4.0,9.66,1,0.95


In [15]:
x = ds[['GRE Score', 'TOEFL Score', 'University Rating', 'CGPA']]
y = ds.iloc[:,-1]

In [16]:
#calculate mean of values in X
M = mean(x.T,axis=1)
print(M)

GRE Score            316.807500
TOEFL Score          107.410000
University Rating      3.087500
CGPA                   8.598925
dtype: float64


In [17]:
c = x-M

In [18]:
V = cov(c.T)
print(V)

[[131.64455514  58.21696742   8.77879073   5.69974242]
 [ 58.21696742  36.83899749   4.82869674   2.99833659]
 [  8.77879073   4.82869674   1.30811404   0.50911685]
 [  5.69974242   2.99833659   0.50911685   0.35559408]]


In [19]:
#calculate eigen values and eigen vectors using eig functions in numpy
eig_vals,eig_vecs = eig(V)
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

In [20]:
#sort eigen value-vector pairs from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)
exp_var_thresh = 0.97  #threshold value of explained variance

In [21]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

In [23]:
num_vec_to_keep = 0
#compute projection matrix based on top eig vectors
for index, percentage in enumerate(cum_var_exp):
  if percentage > exp_var_thresh:
    num_vec_to_keep = index + 1
    break
num_features = x.shape[1]
proj_mat = eig_pairs[0][1].reshape(num_features,1)
for eig_vec_idx in range(1, num_vec_to_keep):
  proj_mat = np.hstack((proj_mat, eig_pairs[eig_vec_idx][1].reshape(num_features,1)))

# Project the data 
pca_data = x.dot(proj_mat)
new_pd = pca_data.join(y)
new_pd.columns=['ReducedFeatures','Chance of Admit']
print(new_pd.head(10))

   ReducedFeatures  Chance of Admit
0       354.659337             0.92
1       338.209124             0.76
2       329.621961             0.72
3       337.622304             0.80
4       327.338148             0.65
5       347.121063             0.90
6       336.274457             0.75
7       321.065862             0.68
8       316.032371             0.50
9       337.663215             0.45
